In [ ]:
'''
Lets make a class for processing data to form edge_list
1> accounts data
2> brand-account and interest account-data
3> audience ages and gender
4> audience brands, interest, cities, countries

'''

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import os

## Preprocess and store

In [37]:
users  = pd.read_csv('./clean/accounts.csv', on_bad_lines='skip', low_memory=False, lineterminator='\n') #We are encoutering some bad lines creating some error so we skip over them
cat=["Id","Gender", "Social network"]
num=['Country id','City id','Followers',"Engagement","Posts","Estimated reach","Avg. views per video", "Avg. comments per post", "Avg. likes per post", "Avg. posts per week"]
users = users[cat+num]
users = users.rename(columns={'City id': 'City','Country id':'Country'})
for i in users.columns:
    users = users.rename(columns={i: i.replace(" ","_")})
#Mapping city    
city_ids = pd.read_csv('../SocialTalk 2023/cities.csv', low_memory=False) 
city_ids['Name'] = city_ids['Name'].str.replace(" ","_")#Replace white space by "_"

users['City'] = users['City'].map(city_ids.set_index('Id')['Name'].to_dict())

#Mapping country
country_ids = pd.read_csv('../SocialTalk 2023/countries.csv', low_memory=False)
country_ids['Name'] = country_ids['Name'].str.replace(" ","_")#Replace white space by "_"

users['Country'] = users['Country'].map(country_ids.set_index('Id')['Name'].to_dict())

# Removing outlier
users.loc[users['Posts'] == 0,'Posts'] = np.nan 
users.loc[users['Avg._views_per_video'] == 0,'Avg._views_per_video'] = np.nan
users.loc[users['Estimated_reach'] == 0,'Estimated_reach'] = np.nan
users.loc[users["Avg._comments_per_post"] == 0 ,"Avg._comments_per_post"] = np.nan
users.loc[users["Avg._likes_per_post"] == 0 ,"Avg._likes_per_post"] = np.nan
users.drop("Avg._posts_per_week",axis=1,inplace = True)

users.rename(columns = {'Id':'Account Id'}, inplace = True)


In [ ]:
'''
q_low = users['Avg._posts_per_week'].quantile(0.01)
q_hi  = users['Avg._posts_per_week'].quantile(0.99)
print(q_low,q_hi)
users[(users['Avg._posts_per_week'] < q_hi) & (users['Avg._posts_per_week'] > q_low)]

'''

In [38]:
users.to_csv('Clean/clean_accounts.csv',index=False)

In [151]:
audience_genders = pd.read_csv('./clean/accounts-audience-genders.csv')
for i in audience_genders.columns[1:]:
    audience_genders.rename(columns = {i:'Audience-Gender-' + i.replace(' ','_')}, inplace = True)
    
audience_ages = pd.read_csv('./clean/accounts-audience-ages.csv')
for i in audience_ages.columns:
    if(i != 'Account Id'):
        audience_ages.rename(columns = {i:'Audience-Age-' + i.replace(' ','_')}, inplace = True)
# Outliers        
audience_ages.loc[audience_ages['Audience-Age-65-_%'] == 0,'Audience-Age-65-_%'] = np.nan

In [152]:
audience_genders.to_csv('Clean/accounts-audience-genders.csv',index=False)
audience_ages.to_csv('Clean/accounts-audience-ages.csv',index=False)

In [140]:
brand_ids = pd.read_csv('../SocialTalk 2023/brands.csv', low_memory=False) # Shape (3792, 2)
account_brands = pd.read_csv('../SocialTalk 2023/accounts-brands.csv', low_memory=False)
interest_ids = pd.read_csv('../SocialTalk 2023/interests.csv', low_memory=False) # Shape (60, 2)
account_interests = pd.read_csv('../SocialTalk 2023/accounts-interests.csv', low_memory=False)

In [141]:
brand_ids['Name'] = brand_ids['Name'].str.replace(" ","_")
interest_ids['Name'] = interest_ids['Name'].str.replace(" ","_")

In [143]:
#brand_ids['label'] = 'Brand:' + brand_ids['Name'].astype(str)
account_brands['Brand Id'] = account_brands['Brand Id'].map(brand_ids.set_index('Id')['Name'].to_dict())
#interest_ids['label'] = 'Interest:' + interest_ids['Name'].astype(str)
account_interests['Interest Id'] = account_interests['Interest Id'].map(interest_ids.set_index('Id')['Name'].to_dict())

In [146]:
for i in account_brands.columns:
    if(i != 'Account Id'):
        account_brands.rename(columns = {i:'Brand'}, inplace = True)
        
for i in account_interests.columns:
    if(i != 'Account Id'):
        account_interests.rename(columns = {i:'Interest'}, inplace = True)

In [147]:
account_brands.to_csv('Clean/accounts-brands.csv',index=False)
account_interests.to_csv('Clean/accounts-interests.csv',index=False)

In [164]:
audience_brands = pd.read_csv('../SocialTalk 2023/accounts-audience-brands.csv')
audience_cities = pd.read_csv('../SocialTalk 2023/accounts-audience-cities.csv')
audience_countries = pd.read_csv('../SocialTalk 2023/accounts-audience-countries.csv')
audience_interests = pd.read_csv('../SocialTalk 2023/accounts-audience-interests.csv')

In [165]:
audience_brands['Brand Id'] = audience_brands['Brand Id'].map(brand_ids.set_index('Id')['Name'].to_dict())
audience_cities['City Id'] = audience_cities['City Id'].map(city_ids.set_index('Id')['Name'].to_dict())
audience_countries['Country Id'] = audience_countries['Country Id'].map(country_ids.set_index('Id')['Name'].to_dict())
audience_interests['Interest Id'] = audience_interests['Interest Id'].map(interest_ids.set_index('Id')['Name'].to_dict())

In [169]:
audience_brands['label'] = 'Audience_Brand_'+audience_brands['Brand Id'].astype(str) 
audience_cities['label'] = 'Audience_City_'+audience_cities['City Id'].astype(str) 
audience_countries['label'] = 'Audience_Country_'+audience_countries['Country Id'].astype(str)
audience_interests['label'] = 'Audience_Interest_'+audience_interests['Interest Id'].astype(str) 

In [173]:
audience_brands.drop(['Brand Id'],axis=1,inplace=True)
audience_cities.drop(['City Id'],axis=1,inplace=True)
audience_countries.drop(['Country Id'],axis=1,inplace=True)
audience_interests.drop(['Interest Id'],axis=1,inplace=True)

In [175]:
audience_brands.to_csv('Clean/Audience/accounts-audience-brands.csv',index=False)
audience_cities.to_csv('Clean/Audience/accounts-audience-cities.csv',index=False) 
audience_countries.to_csv('Clean/Audience/accounts-audience-countries.csv',index=False) 
audience_interests.to_csv('Clean/Audience/accounts-audience-interests.csv',index=False) 

## Graph Class

In [ ]:
# Data given to Graph should be in given format:
'''
1> has Account ID as column for every_data
2> has the preprocessing done before (Mapping..)
3> has only relevant columns
'''

In [20]:
class Graph:
    def __init__(self,data_path,primary_column,sample_accounts=None):
        self.sample_accounts = sample_accounts
        self.path = data_path # path to data folder
        self.Id = primary_column # Account ID
 
    def process_data(self):
        # Clean Folder
        data_files = []
        for i in os.listdir(self.path):
            if(i != '.ipynb_checkpoints' and i != 'Audience'):
                data_files.append(pd.read_csv(self.path + i))
                
        if(isinstance(self.sample_accounts, pd.Series)): # Sampling
            for i in range(len(data_files)):
                #print("Sampling")
                data_files[i] = data_files[i][data_files[i][self.Id].isin(self.sample_accounts)]
                #print(data_files[i].shape)
        
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        edges = []
        for i in range(len(data_files)):
            numeric_columns = data_files[i].select_dtypes(include=numerics).columns
            #print(numeric_columns)
            for j in numeric_columns:
                if(j != self.Id):
                    #print("Bining",j)
                    #print(data_files[i][j].shape)
                    print(j)
                    Quartiles = data_files[i][j].describe()
                    print(Quartiles['min'],Quartiles['25%'],Quartiles['75%'],Quartiles['max'])
                    print()
                    data_files[i][j] = pd.qcut(data_files[i][j],q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High'],duplicates='drop') #Binning Numerical
                    
            data_files[i][self.Id] = 'Account:' + data_files[i][self.Id].astype(str)
            data_files[i] = data_files[i].set_index(self.Id)
            
            for column in data_files[i].columns:
                #print(column)
                series_tuples = [
                    (account, f'{column}:{value}') for account, value in data_files[i][data_files[i][column].notna()][column].items()
                    ]
                edges.extend(series_tuples)
                
        #Audience Folder inside Clean Folder
        audience_files = []
        for i in os.listdir(self.path + 'Audience'):
            if(i != '.ipynb_checkpoints'):
                audience_files.append(pd.read_csv(self.path + 'Audience/' + i))
        
        if(isinstance(self.sample_accounts, pd.Series)): # Sampling
            for i in range(len(audience_files)):
                #print("Sampling")
                audience_files[i] = audience_files[i][audience_files[i][self.Id].isin(self.sample_accounts)]
                #print(audience_files[i].shape)
        
        for i in range(len(audience_files)):
            numeric_columns = audience_files[i].select_dtypes(include=numerics).columns
            #print(numeric_columns)
            for j in numeric_columns:
                if(j != self.Id):
                    #print("Bining",j)
                    #print(audience_files[i][j].shape)
                    print(j)
                    Quartiles = audience_files[i][j].describe()
                    print(Quartiles['min'],Quartiles['25%'],Quartiles['75%'],Quartiles['max'])
                    print()
                    audience_files[i][j] = pd.qcut(audience_files[i][j],q=[0, .25, .75, 1], labels=['Low', 'Medium', 'High'],duplicates='drop') #Binning Numerical
            
            audience_files[i]['label'] = audience_files[i]['label'].astype(str) +":"+ audience_files[i]['%'].astype(str)
            audience_files[i][self.Id] = 'Account:' + audience_files[i][self.Id].astype(str)
            
            edges += [k for k in audience_files[i].set_index(self.Id)['label'].items()]
            
        
                
        temp = pd.DataFrame(edges, columns=['source', 'target'])
        print(temp.shape)
        temp.to_csv(f'edge_list_{len(edges)}.csv',index=False)
        return temp     

In [8]:
users  = pd.read_csv('./Clean/clean_accounts.csv', on_bad_lines='skip', low_memory=False, lineterminator='\n') 

In [9]:
sample_accounts = users[:20000]['Account Id']
sample = pd.read_excel('accounts-examples.xlsx')['Account Id']
sample_accounts = pd.concat([sample,sample_accounts]).drop_duplicates(keep='first')

In [23]:
temp[temp['target'].str.startswith("In")]

,source,target
123720,Account:151,"Interest:Restaurants,_Food_&_Grocery"
123721,Account:151,Interest:Music
123722,Account:151,Interest:Fitness
123723,Account:151,Interest:Music
123724,Account:152,Interest:Tobacco_&_Smoking
...,...,...
240061,Account:1726600,Interest:Business_&_Careers
240062,Account:1726600,"Interest:Friends,_Family_&_Relationships"
240063,Account:2828565,Interest:Electronics_&_Computers
240064,Account:2828565,Interest:Business_&_Careers


In [21]:
a = Graph(data_path = 'Clean/',primary_column='Account Id',sample_accounts = sample_accounts)

In [22]:
temp = a.process_data()

Audience-Age-13-17_%
-0.0970495884823108 0.0247510274808955 0.06455662851437055 0.2911193063644827

Audience-Age-18-24_%
0.0272367313399933 0.20774998529744204 0.42197352672182187 0.7493099509961673

Audience-Age-25-34_%
0.0713037635364394 0.352152305202068 0.4559294592135832 0.6575814790529708

Audience-Age-35-44_%
0.0043692949443617 0.0852033625988615 0.2264374407308873 0.436014

Audience-Age-45-64_%
0.0 0.02147868349358565 0.09379806957554875 0.5425686603327913

Audience-Age-65-_%
6.454527851287678e-05 0.0003355009824912 0.001304172709538825 0.0263511551770804

Followers
0.0 8273.0 58014.5 550332150.0

Engagement
0.0 2813.5 21084.0 1032630249.0

Posts
-5.0 149.0 1329.5 24552763212.0

Estimated_reach
1.0 2762.0 14858.0 38240000.0

Avg._views_per_video
1.0 1103.75 14613.0 40311111.0

Avg._comments_per_post
1.0 10.0 69.0 119545.0

Avg._likes_per_post
-1.0 274.0 1596.5 9809591.0

Audience-Gender-MALE_%
0.0185234541577825 0.2507521407081694 0.6508110078367049 0.9740599714421704

Audience

In [19]:
temp['source']

0             Account:153
1             Account:153
2             Account:154
3             Account:154
4             Account:154
               ...       
719629    Account:2828565
719630    Account:2828565
719631    Account:2828565
719632    Account:2828565
719633    Account:2828565
Name: source, Length: 719634, dtype: object

0          153
1        44207
2        47778
3        42672
4         2126
         ...  
19995    20231
19996    20232
19997    20233
19998    20234
19999    20235
Name: Account Id, Length: 20048, dtype: int64